# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Phạm Nguyễn Quốc Thanh
- MSSV: 21127428
- Lớp: 21CLC08

# Import

In [11]:
import pandas as pd
import numpy as np
from IPython.display import Markdown as md
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
# Import thêm dữ thư viện nếu cần
#1: Male
#2: Female

# Đọc dữ liệu

In [12]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)
X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)
Salary_train = train.iloc[:, [-1]].to_numpy() 
Salary_test = test.iloc[:, [-1]]
first_11_data_train = train.iloc[:, :11].to_numpy()
first_11_data_test = test.iloc[:, :11].to_numpy()
first_3_data = train.iloc[:3, :3]
# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [379]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)

def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel())) 

def format_float(x):
    return '{:.3f}'.format(x)

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [380]:
# Phần code cho yêu cầu 1a
lr_1a = OLSLinearRegression().fit(first_11_data_train, y_train.to_numpy())

# lr_1a = LinearRegression(fit_intercept=False).fit(first_11_data_train, y_train.to_numpy())


In [383]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
y_hat = lr_1a.predict(first_11_data_test)

#Library
# print("MAE: ", mean_absolute_error(y_test.to_numpy().reshape(-1, 1), y_hat.reshape(-1, 1)))
# print(lr_1a.coef_)
# print(lr_1a.intercept_)

#Self
print("MAE: ", mae(y_test.to_numpy().reshape(-1, 1), y_hat.reshape(-1, 1)))
np.set_printoptions(suppress=True, precision=3)
weights = lr_1a.get_params()
print("Weights:\n", weights.reshape(-1, 1))

MAE:  104863.77754032997
Weights:
 [[-22756.513]
 [   804.503]
 [  1294.655]
 [-91781.898]
 [ 23182.389]
 [  1437.549]
 [ -8570.662]
 [   147.858]
 [   152.888]
 [   117.222]
 [ 34552.286]]


In [382]:
first_11_columns = train.columns[:11].to_numpy()
results = zip(first_11_columns, lr_1a.get_params())

display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = " + " + ".join(f"{w:.3f} x {f}" for f, w in results)))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = -22756.513 x Gender + 804.503 x 10percentage + 1294.655 x 12percentage + -91781.898 x CollegeTier + 23182.389 x Degree + 1437.549 x collegeGPA + -8570.662 x CollegeCityTier + 147.858 x English + 152.888 x Logical + 117.222 x Quant + 34552.286 x Domain

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [247]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
train_1b = train
train_1b = train_1b.sample(frac=1)
X_train_1b = train_1b.loc[:, ['conscientiousness', 'agreeableness', 'extraversion', 'openess_to_experience', 'nueroticism']]
y_train_1b = train_1b.iloc[:, -1]
kfold = KFold(n_splits=10, shuffle=False)
models_train = [X_train_1b.loc[:, 'conscientiousness'].to_numpy().reshape(-1, 1), X_train_1b.loc[:, 'agreeableness'].to_numpy().reshape(-1, 1), X_train_1b.loc[:, 'extraversion'].to_numpy().reshape(-1, 1), X_train_1b.loc[:, 'openess_to_experience'].to_numpy().reshape(-1, 1), X_train_1b.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1)]
models_test = [X_test.loc[:, 'conscientiousness'].to_numpy().reshape(-1, 1), X_test.loc[:, 'agreeableness'].to_numpy().reshape(-1, 1), X_test.loc[:, 'extraversion'].to_numpy().reshape(-1, 1), X_test.loc[:, 'openess_to_experience'].to_numpy().reshape(-1, 1), X_test.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1)]

average_maes = [np.abs(cross_val_score(LinearRegression(fit_intercept=False), model, y_train_1b, cv=kfold, scoring='neg_mean_absolute_error')).mean() for model in models_train]
best_model_index = average_maes.index(min(average_maes))
df = pd.DataFrame(list(zip(X_train_1b.columns.to_numpy().flatten(), average_maes)), columns =['Mô hình với 1 đặc trưng', 'MAE'])
df['MAE'] = df['MAE'].apply(format_float)
print(df)
#Nueroticism là đặc trưng tốt nhất

  Mô hình với 1 đặc trưng         MAE
0       conscientiousness  306101.810
1           agreeableness  300684.269
2            extraversion  306958.941
3   openess_to_experience  302940.060
4             nueroticism  299402.877


In [248]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
#Nueroticism columns
my_best_personality_feature_model = OLSLinearRegression().fit(models_train[best_model_index], y_train_1b)

In [249]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
X_Para = models_test[best_model_index]
y_hat = my_best_personality_feature_model.predict(X_Para)
print("MAE: ", mae(y_test, y_hat))
print(my_best_personality_feature_model.w)

MAE:  291019.693226953
[-56546.30375318]


In [244]:
display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = Neuroticism x -56546.304"))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = Neuroticism x -56546.304

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [252]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
train_1c = train
train_1c = train_1c.sample(frac=1)
X_train_1c = train_1c.loc[:, ['Quant', 'Logical',  'English']]
y_train_1c = train_1c.iloc[:, -1]
kfold = KFold(n_splits=10, shuffle=False)
models_train = [X_train_1c.loc[:, 'Quant'].to_numpy().reshape(-1, 1), X_train_1c.loc[:, 'Logical'].to_numpy().reshape(-1, 1), X_train_1c.loc[:, 'English'].to_numpy().reshape(-1, 1)]
models_test = [X_test.loc[:, 'Quant'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Logical'].to_numpy().reshape(-1, 1), X_test.loc[:, 'English'].to_numpy().reshape(-1, 1)]

average_maes = [np.abs(cross_val_score(LinearRegression(fit_intercept=False), model, y_train_1c, cv=kfold, scoring='neg_mean_absolute_error')).mean() for model in models_train]
best_model_index = average_maes.index(min(average_maes))
df = pd.DataFrame(list(zip(X_train_1c.columns.to_numpy().flatten(), average_maes)), columns =['Mô hình với 1 đặc trưng', 'MAE'])
df['MAE'] = df['MAE'].apply(format_float)
print(df)
#Quant là đặc trưng tốt nhất

  Mô hình với 1 đặc trưng         MAE
0                   Quant  118146.442
1                 Logical  120295.723
2                 English  121899.734


In [253]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
my_best_skill_feature_model = OLSLinearRegression().fit(models_train[best_model_index], y_train_1c)

In [256]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
X_Para = models_test[best_model_index]
y_hat = my_best_skill_feature_model.predict(X_Para)
print("MAE: ", mae(y_test, y_hat))
print(my_best_skill_feature_model.w)

MAE:  106819.57761989674
[585.89538084]


In [ ]:
display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = Quant x 585.895"))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = Quant x 585.895

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

## Tìm mô hình

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

Quant (Quantitative) (khả năng định lượng ) là khả năng của 1 người để xử lý và giải quyết dữ liệu "numerical" và "categorical". Logical là khả năng suy luận dựa trên lý lẽ. Chính vì vậy, 2 đặc trưng này được xem là 1 yếu tố cần thiết cho 1 kỹ sư cho nên em quyết định nhân 2 đặc trưng này với nhau và gọi là "Analytical".\
Tiếp theo, em có đặc trưng "English". Ngôn ngữ thường được sử dụng bởi các kỹ sư để có thể trao đổi, giải thích các ý tưởng cho nhau là tiếng Anh. Hơn nữa, hầu hết các bài báo cáo học thuật đều được viết bằng tiếng Anh. Thứ hai, tiếng Anh còn giúp bản thân em giao tiếp với những người đến từ những đất nước khác, hoặc những khách hàng tiềm năng, những người ít nhất cũng kì vọng rằng bản thân ta biết nói tiếng Anh một cách chuẩn xác và lưu loát. Chính vì vậy, đây chính là ngôn ngữ quan trọng nhất mà mọi kỹ sư đều cần phải biết. [The importance of using English Language effectively in Engineering](https://www.gloscol.ac.uk/media/996689/the-importance-of-using-english-language-effectively-in-engineering.pdf). [Is knowing good English necessary for engineers?](https://www.quora.com/Is-knowing-good-English-necessary-for-engineers)\
Kế đến, em có 3 tính cách là "openess to experience", "conscientiousness"và "agreeableness". "Openess to experience" có thể hiểu sơ là người có tính cách cởi mở. Các đặc điểm của tính cách này là: sự sáng tạo, trí tuệ,... "conscientiousness" có thể hiểu sơ là người tận tâm. Tính cách này được thể hiện qua các khía cạnh như: năng lực bản thân, sự ngăn nắp, tính nghiêm túc, thành tích,... "agreeableness"có thể hiểu là người dễ hoà thuận, dễ tính. Cả ba tính cách này trông có vẻ là 1 sự kết hợp khá là kì lạ nhưng nếu thiếu 1 trong 3 tính cách này, có thể sự nghiệp kỹ sư của ta có thể không đi xa được. Bởi vì đây là 1 công việc rất cần tới khả năng tự học các kiến thức chuyên sâu, 1 sự quyết tâm và khả năng tự tổ chức của bản thân. Chính vì vậy, nếu không có "conscientiousness", một kỹ sư tiềm năng có thể sẽ lạc lối. Trong khi đấy, với "openess", điều mà chỉ có thể phát triển hết tiềm năng trong một môi trường đầy sáng tạo mà tính lạ thường hay tính sáng tạo đều rất cần thiết, cũng quan trọng không kém vì 1 kỹ sư mà không dám đổi mới hay khá phá những lĩnh vực khác sẽ không bao giờ đạt được thành tựu gì cả. Tuy nhiên, 2 tính cách trên vẫn có thể sẽ trở nên vô ích khi mà bạn có không có khả năng hoà thuận với những người khác. Các kỹ sư thường sẽ phải làm việc trong một môi trường mà bao gồm rất nhiều cá nhân với những cá tính khác nhau. Tuy nhiên, nói như vậy không có nghĩa là bạn lúc nào cũng phải hoà thuận với người khác. Thỉnh thoảng các kỹ sư cần phải đưa ra những quyết định thông minh, cứng rắn và logic trong nhiều trường hợp, và họ chắc chắn sẽ phải đứng ra lãnh đạo. Chính vì vậy, 3 tính cách nên được gộp thành 1 và được gọi là "Traits".[Should I Become an Engineer?](https://www.truity.com/blog/should-i-become-engineer). [Big 5 Personality Là Gì?](https://glints.com/vn/blog/mo-hinh-big-5-personality-la-gi/). [Here's How Your Personality Type May Affect Your Income](https://www.fool.com/the-ascent/banks/articles/heres-how-your-personality-type-may-affect-your-income/)\
Tiếp theo, em sẽ nói đến đặc trưng "Domain". Theo cá nhân em, ta không thể nào mong chờ 1 mức lương cao ở 1 công việc mà thậm chí bản thân ta còn không có khả năng để làm.\
Các đặc trưng về học thuật như là "10percentage", "12percentage" và "collegeGPA" sẽ được gộp lại thành 1 đặc trưng mới được gọi là "academic_performances" bằng cách lấy trung bình. Để lý giải cho việc này, ta hãy nói đến tính chất công việc của 1 kỹ sư. Để trở thành 1 kỹ sư thành công trong sự nghiệp, trước tiên, họ cần có tính kỷ luật, sự kiên nhẫn và bền bỉ. Ngoài ra, họ còn cần phải có kiến thức chuyên môn vững, khả năng tự học và khả năng thích nghi trước mọi tình huống.\
Đầu tiên, việc duy trì được một GPA ổn định qua nhiều năm có thể được xem như là một điều rất tốt cho 1 kỹ sư tiềm năng. Để làm được như vậy, họ cần phải có tính kỷ luật, sự kiên nhẫn và bền bỉ. Với việc duy trì được một GPA ổn định cũng có thể cho thấy rằng họ có 1 lượng kiến thức ổn định. Tiếp theo, việc học trên trường không phải lúc nào cũng là đủ cả. Để được 1 trung bình ổn định, các kỹ sư cần phải có cho mình một khả năng tự học tốt để tìm kiếm các tài liệu liên quan tới môn học hoặc chuyên ngành. Cuối cùng là khả năng thích nghi, trong 3 đặc trưng trên, ta có thể thấy rõ rệt 2 sự thay đổi rất lớn. Đó là từ lớp 10 lên lớp 12 và từ 12 lên đại học. Rất nhiều người không có khả năng thích nghi khi thay đổi môi trường, ta sẽ có thể thấy sự sụt giảm rõ rệt trong điểm GPA giữa 2 trong 3 đặc trưng.
Kế tiếp là đặc trưng "computerProgramming". Tất nhiên không phải kỹ sư nào cũng đều cần phải biết lập trình máy tính nhưng việc biết làm có thể hỗ trợ thêm cho công việc của bạn.\
Sau đấy, em sẽ nói đến giới tính. Theo Thống kê Cục điều tra dân số Hoa Kỳ năm 2021, phụ nữ làm việc trong ngành kỹ sư cơ khí có khoảng cách thu nhập nhỏ nhất, nhận được 95% mức lương so với nam trong cùng lĩnh vực trung bình. Trong khi đó, phụ nữ làm kỹ sư xây dựng nhận 85% so với mức lương của nam đồng nghiệp.\
Neuroticism (tính nhạy cảm), một đức tính không tốt đẹp gì cả. Vì đặc thù của công việc kỹ sư rất áp lực nên những người đạt điểm thấp trong neuroticism có nghĩa là họ chịu áp lực rất tốt, có thể giúp ích cho họ trong công việc. Tuy nhiên, nói như vậy không có nghĩa là những người có điểm cao trong neuroticism đều sẽ thất bại trong công việc. 

Dựa vào những phân tích trên, em đưa ra quyết định rằng Analytical, Traits, English, Domain, academic_performances và ComputerProgramming sẽ là các đặc trưng quan trọng và cần thiết của mỗi kỹ sư.
$$\text{Salary} = \text{Analytical} \times x + \text{Traits} \times x + \text{English} \times x + \text{Domain} \times x + \text{academic\_performances} \times x + \text{ComputerProgramming} \times x$$
Tiếp theo, đặc trưng "Gender" cũng sẽ quyết định đến 1 phần lương của các kỹ sư nhưng sẽ không quá nhiều.
$$\text{Salary} = \text{Analytical} \times x + \text{Traits} \times x + \text{English} \times x + \text{Domain} \times x + \text{academic\_performances} \times x + \text{ComputerProgramming} \times x+ \text{Gender} \times x$$
Cuối cùng là đặc trưng "neuroticism"
$$\text{Salary} = \text{Analytical} \times x + \text{Traits} \times x + \text{English} \times x + \text{Domain} \times x + \text{academic\_performances} \times x + \text{ComputerProgramming} \times x+ \text{Gender} \times x + \text{Neuroticism} \times x$$

## Thử nghiệm, so sánh các mô hình

In [402]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
train_1d = train
train_1d = train_1d.sample(frac=1)
X_train_1d = train_1d.iloc[:, :-1]    
y_train_1d = train_1d.iloc[:, -1]     
kfold = KFold(n_splits=10, shuffle=False)

analytical = (X_train_1d.loc[:, 'Quant'].to_numpy() * X_train_1d.loc[:, 'Logical'].to_numpy())
analytical_test = (X_test.loc[:, 'Quant'].to_numpy() * X_test.loc[:, 'Logical'].to_numpy())

traits = X_train_1d.loc[:, 'conscientiousness'].to_numpy() * X_train_1d.loc[:, 'agreeableness'].to_numpy() * X_train_1d.loc[:, 'openess_to_experience'].to_numpy()
traits_test = X_test.loc[:, 'conscientiousness'].to_numpy() * X_test.loc[:, 'agreeableness'].to_numpy() * X_test.loc[:, 'openess_to_experience'].to_numpy()

academic_performances = (X_train_1d.loc[:, 'collegeGPA'].to_numpy() + X_train_1d.loc[:, '12percentage'].to_numpy() + X_train_1d.loc[:, '10percentage'].to_numpy())/ 3
academic_performances_test = (X_test.loc[:, 'collegeGPA'].to_numpy() + X_test.loc[:, '12percentage'].to_numpy() + X_test.loc[:, '10percentage'].to_numpy())/ 3

first_model = np.hstack((analytical.reshape(-1, 1), traits.reshape(-1, 1), X_train_1d.loc[:, 'English'].to_numpy().reshape(-1, 1),X_train_1d.loc[:, 'Domain'].to_numpy().reshape(-1, 1), academic_performances.reshape(-1, 1), X_train_1d.loc[:, 'ComputerProgramming'].to_numpy().reshape(-1, 1)))
second_model =  np.hstack((analytical.reshape(-1, 1), traits.reshape(-1, 1), X_train_1d.loc[:, 'English'].to_numpy().reshape(-1, 1),X_train_1d.loc[:, 'Domain'].to_numpy().reshape(-1, 1), academic_performances.reshape(-1, 1), X_train_1d.loc[:, 'ComputerProgramming'].to_numpy().reshape(-1, 1),  X_train_1d.loc[:, 'Gender'].to_numpy().reshape(-1, 1)))
third_model =  np.hstack((analytical.reshape(-1, 1), traits.reshape(-1, 1), X_train_1d.loc[:, 'English'].to_numpy().reshape(-1, 1),X_train_1d.loc[:, 'Domain'].to_numpy().reshape(-1, 1), academic_performances.reshape(-1, 1), X_train_1d.loc[:, 'ComputerProgramming'].to_numpy().reshape(-1, 1),  X_train_1d.loc[:, 'Gender'].to_numpy().reshape(-1, 1),  X_train_1d.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1)))
fourth_model =  np.hstack((analytical.reshape(-1, 1), traits.reshape(-1, 1), X_train_1d.loc[:, 'English'].to_numpy().reshape(-1, 1),X_train_1d.loc[:, 'Domain'].to_numpy().reshape(-1, 1), academic_performances.reshape(-1, 1), X_train_1d.loc[:, 'ComputerProgramming'].to_numpy().reshape(-1, 1),  X_train_1d.loc[:, 'Gender'].to_numpy().reshape(-1, 1),  X_train_1d.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1), X_train_1d.loc[:, 'Degree'].to_numpy().reshape(-1, 1)))

first_model_test = np.hstack((analytical_test.reshape(-1, 1), traits_test.reshape(-1, 1), X_test.loc[:, 'English'].to_numpy().reshape(-1, 1),X_test.loc[:, 'Domain'].to_numpy().reshape(-1, 1), academic_performances_test.reshape(-1, 1), X_test.loc[:, 'ComputerProgramming'].to_numpy().reshape(-1, 1)))
second_model_test =  np.hstack((analytical_test.reshape(-1, 1), traits_test.reshape(-1, 1), X_test.loc[:, 'English'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Domain'].to_numpy().reshape(-1, 1), academic_performances_test.reshape(-1, 1), X_test.loc[:, 'ComputerProgramming'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Gender'].to_numpy().reshape(-1, 1)))
third_model_test =  np.hstack((analytical_test.reshape(-1, 1), traits_test.reshape(-1, 1), X_test.loc[:, 'English'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Domain'].to_numpy().reshape(-1, 1), academic_performances_test.reshape(-1, 1), X_test.loc[:, 'ComputerProgramming'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Gender'].to_numpy().reshape(-1, 1), X_test.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1)))
fourth_model_test =  np.hstack((analytical_test.reshape(-1, 1), traits_test.reshape(-1, 1), X_test.loc[:, 'English'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Domain'].to_numpy().reshape(-1, 1), academic_performances_test.reshape(-1, 1), X_test.loc[:, 'ComputerProgramming'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Gender'].to_numpy().reshape(-1, 1), X_test.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Degree'].to_numpy().reshape(-1, 1)))   

models_train = [first_model, second_model, third_model, fourth_model]
models_test = [first_model_test, second_model_test, third_model_test, fourth_model_test]
average_maes = [np.abs(cross_val_score(LinearRegression(fit_intercept=False), model, y_train, cv=kfold, scoring='neg_mean_absolute_error')).mean() for model in models_train]
best_model_index = average_maes.index(min(average_maes))
print("Best model:", best_model_index + 1)

Best model: 4


In [403]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
my_best_model = OLSLinearRegression().fit(models_train[best_model_index], y_train)



In [404]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
X_para = models_test[best_model_index]
y_hat1 = my_best_model.predict(X_para)
print('MAE trên tập kiểm tra:', mae(y_test, y_hat1))

MAE trên tập kiểm tra: 113035.8690265365


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$